In [1]:
import pickle
from sklearn.manifold import MDS, TSNE
import numpy
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as pyplot
import gensim
from sklearn.cluster import KMeans

In [2]:
main_centers = []
assigned_clusters = []
cluster_centers = []
cluster_weights = []
main_i = []

for i in range(25000):#598927+1
    with open('clusters/article_'+str(i+0)+'.pickle', 'rb') as f:
        this_main_centers, this_assigned_clusters, this_cluster_centers, this_cluster_weights, this_main_i=pickle.load(f)
        main_centers.append(this_main_centers)
        assigned_clusters.append(this_assigned_clusters)
        cluster_centers.append(this_cluster_centers)
        cluster_weights.append(this_cluster_weights)
        main_i.append(this_main_i)

In [3]:
data_clean = []
initial_tags = []

for i in range(25000):
    with open('printable/article_'+str(i+0)+'.pickle', 'rb') as f:
        this_data_clean, this_initial_tags = pickle.load(f)
        data_clean.append(this_data_clean)
        initial_tags.append(this_initial_tags)

In [4]:
#put data in X
X_list = []
for article in main_centers:
    for center in article:
        X_list.append(center)
X = numpy.zeros((len(X_list),300))
for i in range(len(X_list)):
    X[i] = X_list[i]

In [5]:
#word2vec
model = gensim.models.KeyedVectors.load_word2vec_format('cc.fr.300.vec.bin', binary=True)

In [6]:
#kmeans
kmean = KMeans(n_clusters = 400)    
kmean.fit(X)
y = kmean.predict(X)

In [7]:
#determine the closest words
average_vectors = kmean.cluster_centers_
closest_words = []
for i in range(len(average_vectors)):
    class_vectors = X[y==i]
    word_string = ''
    for word_tuple in model.most_similar(positive=[average_vectors[i]], topn=20):
        word_string=word_string+word_tuple[0]+', '
    closest_words.append(word_string)
    

In [8]:
#assign tags to articles
i = 0
final_tags = []
for article in main_centers:
    final_tags.append([])
    for center in article:
        final_tags[-1].append(y[i])
        i=i+1

In [9]:
with open('final_tags/final_tags.pickle', 'wb') as f:
        pickle.dump([average_vectors,closest_words,final_tags,y], f)